# Second Project



Once syntax trees are built, additional analysis and synthesis can be done by evaluating attributes and executing code fragments on tree nodes. We can also walk through the AST to generate a linear N-address code, analogously to LLVM IR. We call this intermediate machine code as uCIR. So, in this second project, you will perform semantic checks on your program, and turn the AST into uCIR. uCIR uses Single Static Assignment (SSA), and can promote stack allocated scalars to virtual registers and remove the load and store operations, allowing better optimizations since values propagate directly to their use sites.  The main thing that distinguishes SSA from a conventional three-address code is that all assignments in SSA are for distinguished name variables.

## Program Checking
First, you will need to define a symbol table that keeps track of
previously declared identifiers.  The symbol table will be consulted
whenever the compiler needs to lookup information about variable and
constant declarations.

Next, you will need to define objects that represent the different
builtin datatypes and record information about their capabilities.

### Type System
Let's define classes that represent types.  There is a general class used to represent all types.  Each type is then a singleton instance of the type class.
```
class uCType(object):
      pass

int_type = uCType("int",...)
float_type = uCType("float",...)
char_type = uCType("char", ...)
```
The contents of the type class is entirely up to you.  However, you will minimally need to encode some information about what operators are supported (+, -, *, etc.), and default values.

Once you have defined the built-in types, you will need to make sure they get registered with any symbol tables or code that checks for type names.

In [3]:
class uCType(object):
    '''
    Class that represents a type in the uC language.  Types 
    are declared as singleton instances of this type.
    '''
    def __init__(self, name, bin_ops=set(), un_ops=set()):
        '''
        You must implement yourself and figure out what to store.
        '''
        self.typename = typename
        self.unary_ops = unary_ops or set()
        self.binary_ops = binary_ops or set()
        self.rel_ops = rel_ops or set()
        self.assign_ops = assign_ops or set()

# Create specific instances of types. You will need to add
# appropriate arguments depending on your definition of uCType
IntType = uCType("int",
                 unary_ops   = {"-", "+", "--", "++", "p--", "p++", "*", "&"},
                 binary_ops  = {"+", "-", "*", "/", "%"},
                 rel_ops     = {"==", "!=", "<", ">", "<=", ">="},
                 assign_ops  = {"=", "+=", "-=", "*=", "/=", "%="}
                 )

FloatType = uCType("float",
                   ...
    )
CharType = uCType("char",
                   ...
    )
ArrayType = uCType("array",
                   unary_ops   = {"*", "&"},
                   rel_ops     = {"==", "!="}
                   )
...

In your type checking code, you will need to reference the
above type objects.   Think of how you will want to access
them.

### Visiting the AST
The following classes for visiting the AST are taken from Python’s ast module:

In [ ]:
class NodeVisitor(object):
    """ A base NodeVisitor class for visiting uc_ast nodes.
        Subclass it and define your own visit_XXX methods, where
        XXX is the class name you want to visit with these
        methods.

        For example:

        class ConstantVisitor(NodeVisitor):
            def __init__(self):
                self.values = []

            def visit_Constant(self, node):
                self.values.append(node.value)

        Creates a list of values of all the constant nodes
        encountered below the given node. To use it:

        cv = ConstantVisitor()
        cv.visit(node)

        Notes:

        *   generic_visit() will be called for AST nodes for which
            no visit_XXX method was defined.
        *   The children of nodes for which a visit_XXX was
            defined will not be visited - if you need this, call
            generic_visit() on the node.
            You can use:
                NodeVisitor.generic_visit(self, node)
        *   Modeled after Python's own AST visiting facilities
            (the ast module of Python 3.0)
    """

    _method_cache = None

    def visit(self, node):
        """ Visit a node.
        """

        if self._method_cache is None:
            self._method_cache = {}

        visitor = self._method_cache.get(node.__class__.__name__, None)
        if visitor is None:
            method = 'visit_' + node.__class__.__name__
            visitor = getattr(self, method, self.generic_visit)
            self._method_cache[node.__class__.__name__] = visitor

        return visitor(node)

    def generic_visit(self, node):
        """ Called if no explicit visitor function exists for a
            node. Implements preorder visiting of the node.
        """
        for c in node:
            self.visit(c)

### Semantic Rules

Finally, you'll need to write code that walks the AST and enforces
a set of semantic rules.  Here is a complete list of everything you'll
need to check:

1.  Names and symbols:

    All identifiers must be defined before they are used.  This includes variables &
    functions.  For example, this kind of code generates an error:
```
       a = 3;              // Error. 'a' not defined.
       int a;
```
    Note: typenames such as "int", "float", and "char" are built-in names that
    should be defined at the start of the program.

2.  Types of literals

    All literal symbols must be assigned a type of "int", "float", "char" or "string".  
    For example:
```
       42;         // Type "int"
       4.2;        // Type "float"
       'x';        // Type "char"
       "forty";    // Type "string"
```
    To do this assignment, check the Python type of the literal value and attach
    a type name as appropriate.

3.  Binary operator type checking

    Binary operators only operate on operands of the same type and produce a
    result of the same type.   Otherwise, you get a type error.  For example:
```
        int a = 2;
        float b = 3.14;

        int c = a + 3;    // OK
        int d = a + b;    // Error.  int + float
        int e = b + 4.5;  // Error.  int = float
```

4.  Unary operator type checking.
```
    Unary operators return a result that's the same type as the operand.
```

5.  Supported operators

    Here are the some examples of operators supported by each type:
```
    int:      binary_ops { +, -, *, /}, unary_ops { +, -}
    float:    rel_ops { ==, !=, <, <=}, assign_ops { +=, -=}
```
    Attempts to use unsupported operators should result in an error. 
    For example:
```
        char[] a = "Hello" + "World";     // OK
        char[] b = "Hello" * "World";     // Error (unsupported op *)
```

6.  Assignment, indexing, etc.

    The left and right hand sides of an assignment operation must be
    declared as the same type. The size os objects must match. The index of an array must be of type int, etc.
    See the examples below:
    ```
    int v[4] = {1, 2, 3};     // Error (size mismatch on initialization)
    float f;
    int j = v[f];             // Error (array index must be of type int)
    j = f;                    // Error (canot assign float to int)
    ```
    However, string literals can be assigned to array of chars. See the example below
    ```
    char c[] = "Susy";        // Ok
    ```
    In this case, the size of ```c``` must be inferred from the initialization

For walking the AST, use the NodeVisitor class. A shell of the code is provided below. Use it as a guide.


In [ ]:
class SymbolTable(object):
    '''
    Class representing a symbol table.  It should provide functionality
    for adding and looking up nodes associated with identifiers.
    '''
    def __init__(self):
        self.symtab = {}
    def lookup(self, a):
        return self.symtab.get(a)
    def add(self, a, v):
        self.symtab[a] = v

class Visitor(NodeVisitor):
    '''
    Program visitor class. This class uses the visitor pattern. You need to define methods
    of the form visit_NodeName() for each kind of AST node that you want to process.
    Note: You will need to adjust the names of the AST nodes if you picked different names.
    '''
    def __init__(self):
        # Initialize the symbol table
        self.symtab = SymbolTable()

        # Add built-in type names (int, float, char) to the symbol table
        self.symtab.add("int",uctype.int_type)
        self.symtab.add("float",uctype.float_type)
        self.symtab.add("char",uctype.char_type)

    def visit_Program(self,node):
        # 1. Visit all of the global declarations
        # 2. Record the associated symbol table
        for _decl in node.gdecls:
            self.visit(_decl)

    def visit_BinaryOp(self, node):
        # 1. Make sure left and right operands have the same type
        # 2. Make sure the operation is supported
        # 3. Assign the result type
        self.visit(node.left)
        self.visit(node.right)
        node.type = node.left.type

    def visit_Assignment(self, node):
        ## 1. Make sure the location of the assignment is defined
        sym = self.symtab.lookup(node.location)
        assert sym, "Assigning to unknown sym"
        ## 2. Check that the types match
        self.visit(node.value)
        assert sym.type == node.value.type, "Type mismatch in assignment"

## Intermediate Representation

At this stage of the project, you are going to turn the AST into an intermediate machine code named uCIR based on Single Static Assignment (SSA). There are a few important parts you'll need to make this work.  Please read 
carefully before beginning:

### Single Static Assignment
The first problem is how to decompose complex expressions into
something that can be handled more simply.  One way to do this is
to decompose all expressions into a sequence of simple assignments
involving binary or unary operations.  

As an example, suppose you had a mathematical expression like this:
```
        2 + 3 * 4 - 5
```
Here is one possible way to decompose the expression into simple
operations:
```
        %1 = 2
        %2 = 3
        %3 = 4
        %4 = %2 * %3
        %5 = %1 + %4
        %6 = 5
        %7 = %5 - %6
```
In this code, the **%n** variables are simply temporaries used while
carrying out the calculation.  A critical feature of SSA is that such
temporary variables are only assigned once (single assignment) and
never reused.  Thus, if you were to evaluate another expression, you
would simply keep incrementing the numbers. For example, if you were
to evaluate **10 + 20 + 30**, you would have code like this:
```
        %8 = 10
        %9 = 20
        %10 = %8 + %9
        %11 = 30
        %12 = %10 + %11
```
SSA is meant to mimic the low-level instructions one might carry out 
on a CPU.  For example, the above instructions might be translated to
low-level machine instructions (for a hypothetical RISC-V CPU) like this:

        addi   t1, zero, 2
        addi   t2, zero, 3
        addi   t3, zero, 4
        mul    t4, t2, t3
        addi   t5, t1, t4
        addi   t6, zero, 5
        sub    s1, t5, t6

Another benefit of SSA is that it is very easy to encode and
manipulate using simple data structures such as tuples. For example,
you could encode the above sequence of operations as a list like this:

       [ 
         ('addi', 't1', zero, 2),
         ('addi', 't2', zero, 3),
         ('addi', 't3', zero, 4),
         ('mul', 't4', 't2', 't3'),
         ('addi', 't5', 't1', 't4'),
         ('addi', 't6', zero, 5),
         ('sub', 's1','t5','t6'),
       ]

### Dealing with Variables
In your program, you are probably going to have some variables that get
used and assigned different values.  For example:
```
       a = 10 + 20;
       b = 2 * a;
       a = a + 1;
```
In "pure SSA", all of your variables would actually be versioned just
like temporaries in the expressions above.  For example, you would
emit code like this:
```
       %1 = 10
       %2 = 20
       a_1 = %1 + %2
       %4 = 2
       b_1 = %4 * a_1
       %5 = 1 
       a_2 = a_1 + %5
       ...
```
To avoid this, we're going to treat declared variables as memory locations and access them using load/store
instructions.  For example:
```
       %1 = 10
       %2 = 20
       %3 = %1 + %2
       store(%3, "a")
       %4 = 2
       %5 = load("a")
       %6 = %4 * %5
       store(%6,"b")
       %7 = load("a")
       %8 = 1
       %9 = %7 + %8
       store(%9, "a")
```

### A Word About Types
At a low-level, CPUs can only operate a few different kinds of 
data such as ints and floats.  Because the semantics of the
low-level types might vary slightly, you'll need to take 
some steps to handle them separately.

In our intermediate code, we're simply going to tag temporary variable
names and instructions with an associated type low-level type.  For
example:

      2 + 3 * 4          (ints)
      2.0 + 3.0 * 4.0    (floats)

The generated intermediate code might look like this:

      ('literal_int', 2, '%1')
      ('literal_int', 3, '%2')
      ('literal_int', 4, '%3')
      ('mul_int', '%2', '%3', '%4')
      ('add_int', '%1', '%4', '%5')

      ('literal_float', 2.0, '%6')
      ('literal_float', 3.0, '%7')
      ('literal_float', 4.0, '%8')
      ('mul_float', '%7', '%8', '%9')
      ('add_float', '%6', '%9', '%10')

### Your Task
Your task is as follows: Write a AST Visitor() class that takes an
uC program and flattens it to a single sequence of SSA code instructions
represented as tuples of the form 
```
       (operation, operands, ..., destination)
```
Your SSA code should only contain the following operators:

#### Variables & Values:
```
      ('alloc_type', varname)              # Allocate on stack (ref by register) a variable of a given type.
      ('global_type', varname, value)      # Allocate on heap a global var of a given type. value is optional.
      ('load_type', varname, target)       # Load the value of a variable (stack/heap) into target (register).
      ('store_type', source, target)       # Store the source/register into target/varname.
      ('literal_type', value, target)      # Load a literal value into target.
      ('elem_type', source, index, target) # Load into target the address of source (array) indexed by index.
      ('get_type', source, target)         # Store into target the address of source (used for pointers).
```

#### Binary Operations:
```
       ('add_type', left, right, target)   # target = left + right
       ('sub_type', left, right, target)   # target = left - right
       ('mul_type', left, right, target)   # target = left * right
       ('div_type', left, right, target)   # target = left / right  (integer truncation)
       ('mod_type', left, right, target)   # target = left % rigth
```

#### Cast Operations:
```
       ('fptosi', fvalue)                   # (int)fvalue == cast float to int 
       ('sitofp', ivalue)                   # (float)ivalue == cast int to float
```

#### Relational/Equality/Logical:
```
       ('oper_type', left, right, target)   # target = left `oper` rigth, where `oper` is:
                                                  lt, le, ge, gt, eq, ne, and, or, not
```

#### Labels & Branches:
```
       ('label', )                                       # Label definition
       ('jump', target)                                  # Jump to a target label
       ('cbranch, expr_test, true_target, false_target)  # Conditional Branch
```

#### Functions & Builtins:
```
       ('define', source)               # Function definition. Source is a function label 
       ('call', source, target)         # Call a function. target is an optional return value
       ('return_type', target)          # Return from function. target is an optional return value
       ('param_type', source)           # source is an actual parameter
       ('read_type', source)            # Read value to source
       ('print_type',source)            # Print value of source
```

### uCIR Examples
Below you find a simple example of the intermediate representation (IR) for the given uC program. More examples are provided in the [uCIR_Examples](./uCIR_Examples.ipynb) notebook.

```
int n = 10;

int foo(int a, int b) {
    return n * (a + b);
}

int main() {
    int c = 2, d = 3;
    int e = foo(c, d);
    return 0;
}

('global_int', '@n', 10)
('define', '@foo')
; function arguments: the value for "a" is passsed by %0, for "b" by %1 & %2 is the slot for return
('alloc_int', '%3')
('alloc_int', '%4')
('store_int', '%0', '%3')
('store_int', '%1', '%4')
('load_int', '%3', '%6')
('load_int', '%4', '%7')
('add_int', '%6', '%7', '%8')
('load_int', '@n', '%9')
('mul_int', '%9', '%8', '%10')
('store_int', '%10', '%2')
('jump', '%5')
('5',)
('load_int', '%2', '%11')
('return_int', '%11')

('define', '@main')
; the main in uC has no arguments, only %0 is reserved for return
('alloc_int', '%2')
('alloc_int', '%3')
('alloc_int', '%4')
('literal_int', 2, '%5')
('store_int', '%5', '%2')
('literal_int', 3, '%6')
('store_int', '%6', '%3')
('load_int', '%2', '%7')
('load_int', '%3', '%8')
('param_int', '%7')
('param_int', '%8')
('call', 'foo', '%9')
('store_int', '%9', '%4')
('literal_int', 0, '%10')
('store_int', '%10', '%0')
('jump', '%1')
('1',)
('load_int', '%0', '%11')
('return_int', '%11')
```

### A note about arrays
The dimensions of an Array in the uC are known at compile time. Then, the type described in the allocation must express the dimension of the same. The initializer_list are always allocated in the heap, either directly in the declaration of the variable, if it is global, or by defining a new temporary, based on the name of the local variable. Examples:

```
int x[] = {1, 2, 3};
void main(){}

('global_int_3', '@x', [1, 2, 3])
('define', '@main')
('1',)
('return_void',)
```

```
int x[2][2];
void main(){
 int y[] = {1, 2, 3};
}

('global_int_2_2', '@x')
('global_int_3', '@.const.y', [1, 2, 3])
('define', '@main')
('alloc_int_2', '%2')
('store_int_2', '@.const.y', '%2')
('1',)
('return_void',)
```

### A note about Pointers
The allocation and operations with pointers in UC follow the same structure used for arrays. The exception 'is that reading the referenced value requires two instructions. see the following examples:

```
int main () {
    int x, y;
    int *r = &x;
    *r = y;
    x = *r;
    return 1;
}

('define', '@main')
('alloc_int', '%2')
('alloc_int', '%3')
('alloc_int_*', '%4')
('get_int_*', '%2', '%4')
('load_int', '%3', '%5')
('store_int_*', '%5', '%4')
('load_int_*', '%4', '%6')
('store_int', '%6', '%2')
('literal_int', 1, '%7')
('store_int', '%7', '%0')
('jump', '%1')
('1',)
('load_int', '%0', '%8')
('return_int', '%8')
```

# Generating Code

Implement the following Node Visitor class so that it creates
a sequence of SSA instructions in the form of tuples.  Use the
above description of the allowed op-codes as a guide.

In [ ]:
class GenerateCode(NodeVisitor):
    '''
    Node visitor class that creates 3-address encoded instruction sequences.
    '''
    def __init__(self):
        super(GenerateCode, self).__init__()

        # version dictionary for temporaries
        self.fname = 'main'  # We use the function name as a key
        self.versions = {self.fname:0}

        # The generated code (list of tuples)
        self.code = []

    def new_temp(self):
        '''
        Create a new temporary variable of a given scope (function name).
        '''
        if self.fname not in self.versions:
            self.versions[self.fname] = 0
        name = "%" + "%d" % (self.versions[self.fname])
        self.versions[self.fname] += 1
        return name

    # You must implement visit_Nodename methods for all of the other
    # AST nodes.  In your code, you will need to make instructions
    # and append them to the self.code list.
    #
    # A few sample methods follow.  You may have to adjust depending
    # on the names of the AST nodes you've defined.

    def visit_Literal(self, node):
        # Create a new temporary variable name 
        target = self.new_temp()

        # Make the SSA opcode and append to list of generated instructions
        inst = ('literal_' + node.type.name, node.value, target)
        self.code.append(inst)

        # Save the name of the temporary variable where the value was placed 
        node.gen_location = target

    def visit_BinaryOp(self, node):
        # Visit the left and right expressions
        self.visit(node.left)
        self.visit(node.right)

        # Make a new temporary for storing the result
        target = self.new_temp()

        # Create the opcode and append to list
        opcode = binary_ops[node.op] + "_"+node.left.type.name
        inst = (opcode, node.left.gen_location, node.right.gen_location, target)
        self.code.append(inst)

        # Store location of the result on the node
        node.gen_location = target

    def visit_PrintStatement(self, node):
        # Visit the expression
        self.visit(node.expr)

        # Create the opcode and append to list
        inst = ('print_' + node.expr.type.name, node.expr.gen_location)
        self.code.append(inst)

    def visit_VarDeclaration(self, node):
        # allocate on stack memory
        inst = ('alloc_' + node.type.name, node.id)
        self.code.append(inst)
        # store optional init val
        if node.value:
            self.visit(node.value)
            inst = ('store_' + node.type.name, node.value.gen_location, node.id)
            self.code.append(inst)

    def visit_LoadLocation(self, node):
        target = self.new_temp()
        inst = ('load_' + node.type.name, node.name, target)
        self.code.append(inst)
        node.gen_location = target

    def visit_AssignmentStatement(self, node):
        self.visit(node.value)
        inst = ('store_' + node.value.type.name, node.value.gen_location, node.location)
        self.code.append(inst)

    def visit_UnaryOp(self, node):
        self.visit(node.left)
        target = self.new_temp()
        opcode = unary_ops[node.op] + "_" + node.left.type.name
        inst = (opcode, node.left.gen_location)
        self.code.append(inst)
        node.gen_location = target

# Writing an Interpreter

Once you've got your compiler emitting intermediate code, you should be able to write a simple interpreter that runs the code.  This can be useful for prototyping the execution environment, testing, and other tasks involving the generated code.

You can think the Interpreter as a kind of stack machine, which means that most instructions take their operands from the stack, and place results back on the stack.

You can define a memory model that consists of a program memory (the code), a dictionary to hold references (indexes) to vars, labels & registers in the memory (M). All the data areas of M are divided into cells, and each cell can hold a single value. The actual size of a cell must be large enough to hold single values (int, char, bool and ref) or any element of string (chars) and arrays. For simplicity, you can use a separate dictionary to hold the indexes of globals vars and constants. These vars & constants will be previously stored at begining of the memory by the interpreter before start running the program.

You use a program counter “pc” to fetches instructions from the code. In this model, the M stack does not act as a function stack for holding function linkage information but only data. You can use auxiliares stack and dictionaries to holding these informations.

Your task is to extend the Interpreter class below so that it can run the code you generated above.  The comments and docstrings in the class gives you more details.

In [8]:
class Interpreter(object):
    """
    Runs an interpreter on the SSA intermediate code generated for
    your compiler.   The implementation idea is as follows.  Given
    a sequence of instruction tuples such as:

         code = [ 
              ('literal_int', 1, '%1'),
              ('literal_int', 2, '%2'),
              ('add_int', '%1', '%2, '%3')
              ('print_int', '%3')
              ...
         ]

    The class executes methods self.run_opcode(args).  For example:

             self.run_literal_int(1, '%1')
             self.run_literal_int(2, '%2')
             self.run_add_int('%1', '%2', '%3')
             self.run_print_int('%3')

    For builtin function declarations, allow specific Python modules
    (e.g., print, input, etc.) to be registered with the interpreter.
    """
    
    def __init__(self):
        global M
        M = 10000 * [None]       # Memory for holding data

        self.globals = {}        # Dictionary of address of global vars & constants
        self.vars = {}           # Dictionary of address of local vars relative to sp

        self.offset = 0          # offset (index) of local & global vars. Note that
                                 # each instance of var has absolute address in Memory
        self.stack = []          # Stack to save offset of vars between calls
        self.sp = []             # Stack to save & restore the last offset

        self.params = []         # List of parameters from caller (address)
        self.result = None       # Result Value (address) from the callee

        self.registers = []      # Stack of register names (in the caller) to return value
        self.returns = []        # Stack of return addresses (program counters)

        self.pc = 0              # Program Counter
        self.start = 0           # PC of the main function

    def _alloc_reg(self, target):
        if target not in self.vars:
            self.vars[target] = self.offset
            self.offset += 1

    def _extract_operation(self, source):
        # Extract the operation & their modifiers
        _modifier = {}
        _aux = source.split('_')
        # ...
        return (_opcode, _modifier)

    def run(self, ircode):
        '''
        Run intermediate code in the interpreter.  ircode is a list
        of instruction tuples.  Each instruction (opcode, *args) is 
        dispatched to a method self.run_opcode(*args)
        '''
        self.pc = 0
        # First, store the global vars & constants in Memory
        # and hold their offsets in self.globals dictionary
        # Also, set the start pc to the main function entry
        self.pc = self.start
        while True:
            try:
                op = ircode[self.pc]
            except IndexError:
                break
            self.pc += 1
            if not op[0].isdigit():
                opcode, modifier = self._extract_operation(op[0])
                if hasattr(self, "run_" + opcode):
                    if not modifier:
                        getattr(self, "run_" + opcode)(*op[1:])
                    else:
                        getattr(self, "run_" + opcode + '_')(*op[1:], **modifier)
                else:
                    print("Warning: No run_" + opcode + "() method")

    # YOU MUST IMPLEMENT methods for different opcodes.  A few sample
    # opcodes are shown below to get you started.

    def run_jump(self, target):
        self.pc = self.vars[target]

    def run_cbranch(self, expr_test, true_target, false_target):
        if M[self.vars[expr_test]]:
            self.pc = self.vars[true_target]
        else:
            self.pc = self.vars[false_target]

    # load literals into registers
    def run_literal_int(self, value, target):
        self._alloc_reg(target)
        M[self.vars[target]] = value

    run_literal_float = run_literal_int
    run_literal_char = run_literal_int

    # perform binary operations
    def run_add_int(self, left, right, target):
        self._alloc_reg(target)
        M[self.vars[target]] = M[self.vars[left]] + M[self.vars[right]]

    run_add_float = run_add_int
    run_add_string = run_add_int

    def run_alloc_int_(self, varname, **kwargs):
    _size = 1
    for arg in kwargs.values():
        if arg.isdigit():
            _size *= int(arg)
    self.vars[varname] = self.offset
    M[self.offset:self.offset + _size] = _size * [0]
    self.offset += _size

    run_alloc_float_ = run_alloc_int_
    run_alloc_char_ = run_alloc_int_

    def run_store_int_(self, source, target, **kwargs):
        _ref = 0
        _size = 1
        for arg in kwargs.values():
            if arg.isdigit():
                _size *= int(arg)
            elif arg == '*':
                _ref += 1
        if _ref == 0:
            self._store_multiple_values(_size, target, source)
        elif _size == 1 and _ref == 1:
            self._store_deref(target, self._get_value(source))
        # ...

    run_store_float_ = run_store_int_
    run_store_char_ = run_store_int_

    # Load/stores
    def run_load_int(self, varname, target):
        self._alloc_reg(target)
        M[self.vars[target]] = self._get_value(varname)

    run_load_float = run_load_int
    run_load_char = run_load_int
    run_load_bool = run_load_int
    
    def run_call(self, source, target):
        self._alloc_reg(target)
        self.registers.append(target)
        # save the return pc
        self.returns.append(self.pc)
        # jump to the function
        self.pc = self.globals[source]

    def run_elem_int(self, source, index, target):
        self._alloc_reg(target)
        _aux = self._get_address(source)
        _idx = self._get_value(index)
        _address = _aux + _idx
        self._store_value(target, _address)

    run_elem_float = run_elem_int
    run_elem_char = run_elem_int

    def run_param_int(self, source):
        self.params.append(self.vars[source])

    run_param_float = run_param_int
    run_param_char = run_param_int